In [6]:
# Download from Google Drive
!curl -L -o tennis_court_det_dataset.zip "https://drive.usercontent.google.com/download?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu&export=download&authuser=0&confirm=t&uuid=3077628e-fc9b-4ef2-8cde-b291040afb30&at=APZUnTU9lSikCSe3NqbxV5MVad5T%3A1708243355040"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
  0 6919M    0 1978k    0     0   503k      0  3:54:33  0:00:03  3:54:30  503k
  0 6919M    0 29.5M    0     0  6137k      0  0:19:14  0:00:04  0:19:10 6139k
  0 6919M    0 56.3M    0     0  9739k      0  0:12:07  0:00:05  0:12:02 11.6M
  1 6919M    1 84.5M    0     0  12.2M      0  0:09:26  0:00:06  0:09:20 18.3M
  1 6919M    1  109M    0     0  13.8M      0  0:08:19  0:00:07  0:08:12 23.8M
  2 6919M    2  139M    0     0  15.5M      0  0:07:24  0:00:08  0:07:16 27.4M
  2 6919M    2  166M    0     0  16.7M      0  0:06

In [7]:
# Unzip the data
!unzip tennis_court_det_dataset.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [35]:
# Import Necessary Libraries
import torch 
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import json
import numpy as np
import cv2

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create Dataset

In [44]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w 
        kps[1::2] *= 224.0 / h 

        return img, kps

In [38]:
train_dataset = KeypointsDataset("tennis_court_det_dataset/data/images","tennis_court_det_dataset/data/data_train.json")
val_dataset = KeypointsDataset("tennis_court_det_dataset/data/images","tennis_court_det_dataset/data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

# Create Model

In [39]:
model = models.resnet50(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

c:\Users\arjun\OneDrive\Documents\Visual Studio 2022\Python\Project_Tennis\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\arjun\OneDrive\Documents\Visual Studio 2022\Python\Project_Tennis\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [40]:
model = model.to(device)

# Train Model

In [41]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [45]:
epochs=20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")

Epoch 0, iter 0, loss: 11056.982421875
Epoch 0, iter 10, loss: 10463.5185546875
Epoch 0, iter 20, loss: 10024.4091796875
Epoch 0, iter 30, loss: 10430.1943359375
Epoch 0, iter 40, loss: 9619.4404296875
Epoch 0, iter 50, loss: 8940.126953125
Epoch 0, iter 60, loss: 8625.4287109375
Epoch 0, iter 70, loss: 9088.9404296875
Epoch 0, iter 80, loss: 8128.88037109375
Epoch 0, iter 90, loss: 7791.2646484375
Epoch 0, iter 100, loss: 7659.5205078125
Epoch 0, iter 110, loss: 8142.58154296875
Epoch 0, iter 120, loss: 6366.09912109375
Epoch 0, iter 130, loss: 8089.1279296875
Epoch 0, iter 140, loss: 6512.33447265625
Epoch 0, iter 150, loss: 5754.71630859375
Epoch 0, iter 160, loss: 6205.08203125
Epoch 0, iter 170, loss: 5741.7236328125
Epoch 0, iter 180, loss: 5566.11279296875
Epoch 0, iter 190, loss: 5568.7880859375
Epoch 0, iter 200, loss: 5081.13037109375
Epoch 0, iter 210, loss: 4728.57470703125
Epoch 0, iter 220, loss: 4588.61865234375
Epoch 0, iter 230, loss: 4387.7939453125
Epoch 0, iter 240,

KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), "keypoints_model.pth")